# **Árvores de Decisão para Regressão — Módulo 4, Notebook 3/4**

---

## Índice

1. [Introdução](#introducao)
2. [Intuição e Motivação](#intuicao)
3. [Construção Manual: Splits Passo-a-Passo](#splits-manuais)
4. [Formulação Matemática](#formulacao)
5. [Implementação com Sklearn](#sklearn)
6. [Visualização da Árvore](#visualizacao-arvore)
7. [Feature Importance](#feature-importance)
8. [Hiperparâmetros e Grid Search](#hiperparametros)
9. [Diagnóstico de Resíduos](#residuos)
10. [Comparação com Outros Modelos](#comparacao)
---

<a id='introducao'></a>
## **Introdução**
Na classificação, cada nó busca uma pergunta que **reduza impureza** (Gini, Entropia) de forma mais agressiva. Já na regressão, o objetivo muda: agora queremos nós (segmentos) onde os valores reais **fiquem concentrados** e o erro seja baixo.

Em vez de medir mistura de classes, medimos quão dispersos são os valores numéricos dentro de cada região.

Ou seja, para regressão buscamos tipicamente:
- Redução da Soma dos Erros Quadráticos (SSE)
- Redução da Variância
- Mean Squared Error (MSE) dentro dos nós

A ideia continua ser escolher o melhor split local, construir recursivamente até parar por algum critério.

<a id='intuicao'></a>
## **Intuição e Motivação**

Vamos manter a ideia da predição do preço de uma casa usando:
- Área (m²)
- Número de quartos
- Idade do imóvel

Uma árvore de regressão pensa em "segmentar" o espaço em blocos onde o **preço médio** dentro de cada bloco seja uma boa aproximação. Em vez de devolver uma classe na folha, ela devolve um **valor (geralmente a média ou mediana dos valores naquele nó)**.

Perguntas sucessivas podem criar regiões cada vez mais homogêneas em termos de preço, ou seja, se segmentarmos demais, memorizamos ruído (overfitting). Se segmentarmos de menos, perdemos estrutura (underfitting).

Vamos construir um exemplo sintético com relação não-linear para visualizar.

In [ ]:

# =======================================================
# DADOS E VISUALIZAÇÃO INICIAL
# =======================================================

# Imports principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline


In [ ]:
# Gerar dados sintéticos (não-linear + interação)
np.random.seed(42)
N = 400
X1 = np.random.uniform(0, 10, N)          # Ex: área
X2 = np.random.uniform(-5, 5, N)           # Ex: idade (centrada)
X3 = np.random.uniform(0, 1, N)            # Ex: fator de localização (score)

# Relação verdadeira (não linear)
# Inclui termo senoidal e interação X1*X3
y = 15 + 4*X1 - 2.5*X2 + 10*X3 + 3*np.sin(X1) + 5*X1*X3 + np.random.normal(0, 2.0, N)

X = pd.DataFrame({'Area': X1, 'Idade': X2, 'LocalScore': X3})
df = X.copy()
df['Preco'] = y

print(df.head())
print(f"Shape: {df.shape}")

In [ ]:
# Visualização
plt.figure(figsize=(7,5))
sc = plt.scatter(df['Area'], df['Preco'], c=df['LocalScore'], cmap='viridis', alpha=0.7)
plt.colorbar(sc, label='LocalScore')
plt.xlabel('Área (m²)')
plt.ylabel('Preço')
plt.title('Dados Sintéticos - Regressão')
plt.show()


<a id='splits-manuais'></a>

## **Construção Manual: Splits Passo-a-Passo**Vamos construir uma árvore de decisão manualmente, calculando SSE e ganho de informação em cada split. Começaremos com um split de `LocalScore > 0.5` criando duas folhas.


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def sse(y):
    return ((y - y.mean())**2).sum() # Soma dos erros ao quadrado

# split por LocalScore > 0.5
threshold_local = 0.5
grupo_esq = df[df['LocalScore'] <= threshold_local]
grupo_dir = df[df['LocalScore'] > threshold_local]

baseline_sse = sse(df['Preco'])
baseline_mse = ((df['Preco'] - df['Preco'].mean())**2).mean()

split_sse = sse(grupo_esq['Preco']) + sse(grupo_dir['Preco'])
gain_1 = baseline_sse - split_sse
split_mse = split_sse / len(df)

pred_manual1 = np.where(df['LocalScore'] > threshold_local, grupo_dir['Preco'].mean(), grupo_esq['Preco'].mean())
rmse1 = mean_squared_error(df['Preco'], pred_manual1)
mae1 = mean_absolute_error(df['Preco'], pred_manual1)
r2_1 = r2_score(df['Preco'], pred_manual1)

print('=== Árvore ===')
print(f'Tamanho total: {len(df)} | Esq: {len(grupo_esq)} | Dir: {len(grupo_dir)}')
print(f'Média Global: {df['Preco'].mean():.3f}')
print(f'Média Esq (LS<=0.5): {grupo_esq['Preco'].mean():.3f}')
print(f'Média Dir (LS>0.5): {grupo_dir['Preco'].mean():.3f}')
print('\nSSE Baseline:', f'{baseline_sse:.2f}')
print('SSE Pós-Split:', f'{split_sse:.2f}')
print('Gain (redução SSE):', f'{gain_1:.2f}')
print('\nMSE Baseline:', f'{baseline_mse:.3f}')
print('MSE Pós-Split:', f'{split_mse:.3f}')
print('\nMétricas predição (2 folhas):')
print(f'RMSE: {rmse1:.3f} | MAE: {mae1:.3f} | R²: {r2_1:.3f}')

### Como Interpretar os Resultados do Split Manual

**SSE (Sum of Squared Errors):**
- **Baseline SSE:** Erro total quando usamos apenas a média global (sem splits)
- **Pós-Split SSE:** Erro total após dividir em dois grupos
- **Gain (Redução):** Quanto o split reduziu o erro. Maior = melhor split

**Grupos criados:**
- Cada grupo tem sua própria média como predição
- Grupos mais homogêneos (baixa variância interna) = predições mais precisas

**Métricas finais:**
- **R²:** Percentual da variância explicada (0 a 1, maior = melhor)
- **RMSE:** Erro médio em unidades originais (menor = melhor)
- Comparar sempre com baseline (média global) para avaliar ganho real

In [ ]:
# Segundo nível: adicionar split por Area > 5 dentro de cada lado
threshold_area = 5
L_esq = df[(df['LocalScore'] <= threshold_local) & (df['Area'] <= threshold_area)]
L_esq_2 = df[(df['LocalScore'] <= threshold_local) & (df['Area'] > threshold_area)]
L_dir = df[(df['LocalScore'] > threshold_local) & (df['Area'] <= threshold_area)]
L_dir_2 = df[(df['LocalScore'] > threshold_local) & (df['Area'] > threshold_area)]

sse_4 = sse(L_esq['Preco']) + sse(L_esq_2['Preco']) + sse(L_dir['Preco']) + sse(L_dir_2['Preco'])
gain_total_2 = baseline_sse - sse_4
gain_incremental_2 = split_sse - sse_4

mean_L_esq = L_esq['Preco'].mean()
mean_L_esq_2 = L_esq_2['Preco'].mean()
mean_L_dir = L_dir['Preco'].mean()
mean_L_dir_2 = L_dir_2['Preco'].mean()

def predict_row_manual2(row):
    if row['LocalScore'] <= threshold_local:
        if row['Area'] <= threshold_area:
            return mean_L_esq
        else:
            return mean_L_esq_2
    else:
        if row['Area'] <= threshold_area:
            return mean_L_dir
        else:
            return mean_L_dir_2

pred_manual2 = df.apply(predict_row_manual2, axis=1).values
rmse2 = mean_squared_error(df['Preco'], pred_manual2)
mae2 = mean_absolute_error(df['Preco'], pred_manual2)
r2_2 = r2_score(df['Preco'], pred_manual2)

print('\n=== Árvore 2 ===')
print('Tamanhos folhas:', len(L_esq), len(L_esq_2), len(L_dir), len(L_dir_2))
print(f'SSE 1º nível: {split_sse:.2f}')
print(f'SSE 2º nível: {sse_4:.2f}')
print(f'Gain acumulado total: {gain_total_2:.2f}')
print(f'Gain incremental segundo nível: {gain_incremental_2:.2f}')
print('\nMédias por folha:')
print(f'  (LS<=0.5, A<=5): {mean_L_esq:.3f}')
print(f'  (LS<=0.5, A>5): {mean_L_esq_2:.3f}')
print(f'  (LS>0.5, A<=5): {mean_L_dir:.3f}')
print(f'  (LS>0.5, A>5): {mean_L_dir_2:.3f}')
print('\nMétricas predição (4 folhas):')
print(f'RMSE: {rmse2:.3f} | MAE: {mae2:.3f} | R²: {r2_2:.3f}')
print('\nEvolução RMSE:', f'{rmse1:.3f} -> {rmse2:.3f}')
print('Evolução R²  :', f'{r2_1:.3f} -> {r2_2:.3f}')

<a id='formulacao'></a>
## **Formulação Matemática**

A árvore de regressão é MUITO parecida com a da classificação, só trocamos a métrica de impureza de classe por uma métrica de dispersão de valores.

**Entrada**

1. Conjunto de treinamento de n amostras em pares $ D=\{(x^{(i)}, y^{(i)})\}_{i=1}^n $

    onde:

    - $ x^{(i)} $: vetor de d features ($ x^{(i)} = (x^{(i)}_1, x^{(i)}_2, ..., x^{(i)}_d) $).

    - $ y^{(i)} $: rótulo numérico contínuo, ou seja, $ y^{(i)} \in \mathbb{R} $.

**Processamento**

O algoritmo constrói a árvore de forma recursiva, procurando em cada nó a melhor divisão possível. A "melhor" divisão é aquela que torna os subconjuntos resultantes o mais homogêneos possível em relação ao valor de $ y $.

**1. Critério de 'Impureza' para Regressão (Dispersão)**

Em vez de medir a mistura de classes (Gini/Entropia), medimos a dispersão dos valores numéricos dentro de um nó. O critério mais comum é a ja conhecida **Soma dos Erros Quadráticos (SSE - Sum of Squared Errors)**, que é minimizada quando os valores em um nó estão próximos uns dos outros.

Para um nó $ S $, o SSE é calculado em relação à média dos valores ($ \bar{y}_S $) daquele nó:

$$SSE(S) = \sum_{i \in S} (y^{(i)} - \bar{y}_S)^2$$

Um nó com baixo SSE é considerado "puro" no contexto da regressão.

**2. Ganho na Divisão (Redução de Variância)**

De forma análoga ao *Information Gain*, o algoritmo escolhe a divisão que maximiza a **redução do SSE**. Para uma feature $ A $ que divide o nó $ S $ em subconjuntos $ S_L $ (esquerda) e $ S_R $ (direita), o ganho é:
$$ \text{Redução de SSE} = SSE(S) - (SSE(S_L) + SSE(S_R))$$

O algoritmo testa todas as features e todos os possíveis pontos de corte para encontrar a divisão que resulta na maior redução de SSE.

**3. Algoritmo de Construção**

O processo é o mesmo da classificação, mas usando a métrica de regressão:

```
função BuildTree(S, Features):
    se critério de parada for atingido (ex: profundidade máxima):
        retorna folha com o valor médio de y em S
    
    seleciona feature A e threshold t que maximizam a Redução de SSE
    cria nó com o teste (A <= t)
    
    S_L, S_R = divide S baseado no teste
    
    adiciona subárvore esquerda BuildTree(S_L, Features)
    adiciona subárvore direita BuildTree(S_R, Features)
```

**4. Critérios de Parada**

Idênticos aos da classificação, usados para regularizar a árvore e evitar overfitting:
- Profundidade máxima da árvore (`max_depth`).
- Número mínimo de amostras para dividir um nó (`min_samples_split`).
- Número mínimo de amostras em um nó folha (`min_samples_leaf`).

**Saída - Regressão**

- **Estrutura da Árvore**: Um conjunto de nós com regras de divisão.
- **Predição**: Para uma nova amostra, ela percorre a árvore de acordo com as regras até chegar a um nó folha. O valor da predição é o **valor médio** de $ y $ das amostras de treinamento que caíram naquela folha durante a construção da árvore.

**Poda (Pruning)**

O conceito de pré-poda (usando critérios de parada) e pós-poda para controlar a complexidade e evitar overfitting é exatamente o mesmo aplicado às árvores de classificação.

In [ ]:
# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape)


<a id='sklearn'></a>
## **Implementação com Sklearn**

Vamos treinar uma árvore de regressão simples e observar métricas:
- RMSE
- MAE
- R²

In [ ]:
# Treino simples
reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_train, y_train)

pred = reg.predict(X_test)
rmse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)
print(f"RMSE: {rmse:.3f} | MAE: {mae:.3f} | R²: {r2:.3f}")

# Visualização: real vs predito
plt.figure(figsize=(6,5))
plt.scatter(y_test, pred, alpha=0.6)
plt.xlabel('Real')
plt.ylabel('Predito')
plt.title('Real vs Predito - Árvore Regressão')
lims = [min(y_test.min(), pred.min()), max(y_test.max(), pred.max())]
plt.plot(lims, lims, 'r--') 
plt.show()


### Interpretando o Gráfico Real vs Predito

**O que procurar:**
- **Pontos na diagonal vermelha:** Predição perfeita (real = predito)
- **Dispersão em torno da diagonal:** Indica qualidade do ajuste
  - Dispersão pequena = bom modelo
  - Dispersão grande = modelo fraco
- **Padrões sistemáticos:** Se pontos formam curva, modelo não captura toda a estrutura
- **Outliers afastados:** Pontos muito distantes da diagonal podem indicar dados anômalos

**Métricas complementares:**
- **RMSE < 3.0:** Excelente para este problema
- **R² > 0.90:** Modelo explica mais de 90% da variância

<a id='visualizacao-arvore'></a>
## **Visualização da Árvore**

Uma das grandes vantagens das árvores de decisão é a interpretabilidade. Podemos visualizar graficamente a estrutura completa da árvore e extrair as regras de decisão em formato textual.

In [ ]:
# =======================================================
# VISUALIZAÇÃO DA ESTRUTURA DA ÁRVORE
# =======================================================

# Visualizar árvore completa
plt.figure(figsize=(20, 10))
plot_tree(reg, 
          feature_names=['Area', 'Idade', 'LocalScore'],
          filled=True, 
          rounded=True, 
          fontsize=10)
plt.title('Estrutura da Árvore de Regressão (sem poda) (que lindo!)')
plt.tight_layout()
plt.show()

# Regras textuais
print("\n=== Regras da Árvore (Texto) ===")
tree_rules = export_text(reg, feature_names=['Area', 'Idade', 'LocalScore'])
print(tree_rules)

### Visualizando a Predição em Degraus

Uma característica fundamental das Árvores de Decisão é que suas predições são feitas em "degraus". Isso significa que a árvore divide o espaço de features em regiões retangulares, e dentro de cada região, a predição é um valor constante (a média dos valores de treinamento naquela região).

Vamos visualizar isso com um exemplo simples usando apenas uma feature.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

N_viz = 400
X1_viz = np.random.uniform(0, 10, N_viz) # Ex: área
X_1d = X1_viz.reshape(-1, 1)
noise_viz = np.random.normal(0, 2.0, N_viz)
y_1d = 15 + 4*X1_viz + 3*np.sin(X1_viz) + noise_viz 


X_train_1d, X_test_1d, y_train_1d, y_test_1d = train_test_split(X_1d, y_1d, test_size=0.25, random_state=42)

# Treinar uma Árvore de Decisão de Regressão com profundidade limitada
tree_reg_1d = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_reg_1d.fit(X_train_1d, y_train_1d)

# Gerar pontos para plotar a função de degraus
X_plot_1d = np.linspace(X_1d.min(), X_1d.max(), 500).reshape(-1, 1)
y_plot_1d = tree_reg_1d.predict(X_plot_1d)

plt.figure(figsize=(10, 6))
plt.scatter(X_1d, y_1d, s=20, edgecolor="black", c="darkorange", label="Dados Reais")
plt.plot(X_plot_1d, y_plot_1d, color="cornflowerblue", label="Predição da Árvore (degraus)", linewidth=2)

plt.xlabel("Feature Única (Ex: Área)")
plt.ylabel("Target (Ex: Preço)")
plt.title("Visualização da Predição em Degraus de uma Árvore de Regressão")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

<a id='feature-importance'></a>
## **Feature Importance**

Árvores de decisão calculam automaticamente a importância de cada feature baseada em quanto ela contribui para reduzir o SSE nos splits. Features que aparecem mais cedo na árvore (próximo à raiz) e que produzem maiores ganhos tendem a ter maior importância.

In [ ]:
# Definir grid de hiperparâmetros e executar Grid Search
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'criterion': ['squared_error', 'friedman_mse']
}

reg_base = DecisionTreeRegressor(random_state=42)
grid = GridSearchCV(reg_base, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=0)
grid.fit(X_train, y_train)

print('Melhores parâmetros:', grid.best_params_)
print('Melhor R² (CV):', f'{grid.best_score_:.4f}')

best = grid.best_estimator_
pred_best = best.predict(X_test)

print(f'\nR² teste: {r2_score(y_test, pred_best):.4f}')
print(f'RMSE teste: {np.sqrt(mean_squared_error(y_test, pred_best)):.3f}')
print(f'MAE teste: {mean_absolute_error(y_test, pred_best):.3f}')


In [ ]:
# =======================================================
# IMPORTÂNCIA DAS FEATURES
# =======================================================

# Usar modelo otimizado do Grid Search
importances = best.feature_importances_
feature_names = ['Area', 'Idade', 'LocalScore']

# Ordenar por importância
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(8, 5))
plt.bar(range(len(importances)), importances[indices], color='steelblue', edgecolor='black')
plt.xticks(range(len(importances)), [feature_names[i] for i in indices])
plt.xlabel('Feature')
plt.ylabel('Importância')
plt.title('Feature Importance - Árvore de Decisão')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n=== Feature Importance ===")
for i in indices:
    print(f"{feature_names[i]:12s}: {importances[i]:.4f}")

print("\n--- Interpretação ---")
print("Importância indica quanto cada feature contribui para reduzir SSE nos splits.")
print(f"'{feature_names[indices[0]]}' é a mais importante (usada nos splits principais).")
print(f"Se alguma feature tem importância ~0, ela não foi usada em nenhum split.")

<a id='hiperparametros'></a>
## **Hiperparâmetros e Grid Search**

Vamos buscar combinação que maximize R² com validação cruzada.

In [ ]:
param_grid = {
    'max_depth': [3,5,7,None],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1,2,5],
    'criterion': ['squared_error','friedman_mse']
}
reg_base = DecisionTreeRegressor(random_state=42)
grid = GridSearchCV(reg_base, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=0)
grid.fit(X_train, y_train)
print('Melhores parâmetros:', grid.best_params_)
print('Melhor R2 CV:', grid.best_score_)

best = grid.best_estimator_
pred_best = best.predict(X_test)
print('R2 teste:', r2_score(y_test, pred_best))
print('RMSE teste:', mean_squared_error(y_test, pred_best))

### Interpretando Resultados do Grid Search

**Melhores parâmetros encontrados:**
- **max_depth:** Controla profundidade máxima. Valores menores = mais generalização
- **min_samples_split:** Mínimo para dividir nó. Valores maiores = árvore mais conservadora
- **min_samples_leaf:** Mínimo em folha. Evita folhas com poucos pontos (reduz overfitting)
- **criterion:** Métrica de qualidade do split
  - `squared_error`: SSE padrão
  - `friedman_mse`: MSE com penalização (melhora em alguns casos)

**Melhor score CV vs Teste:**
- Se R² CV ≈ R² teste: Modelo generalizando bem
- Se R² CV >> R² teste: Possível overfitting (revisar hiperparâmetros)
- Se R² CV << R² teste: Sorte no split (executar múltiplas rodadas)

<a id='residuos'></a>
## **Diagnóstico de Resíduos**

Resíduos = real - predito. Devem estar centrados em zero e sem padrão claro.

In [ ]:
res = y_test - pred_best
fig, axes = plt.subplots(1,3, figsize=(14,4))

# Scatter residuos vs predito
axes[0].scatter(pred_best, res, alpha=0.6)
axes[0].axhline(0, color='red', linestyle='--')
axes[0].set_xlabel('Predito')
axes[0].set_ylabel('Resíduo')
axes[0].set_title('Resíduo vs Predito')

# Histograma
axes[1].hist(res, bins=20, edgecolor='k')
axes[1].set_xlabel('Resíduo')
axes[1].set_ylabel('Freq')
axes[1].set_title('Distribuição Resíduos')

# QQ plot
import scipy.stats as stats
stats.probplot(res, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot')

plt.tight_layout()
plt.show()

### Interpretando a Análise de Resíduos

**Gráfico 1: Resíduo vs Predito**
- **Ideal:** Pontos distribuídos aleatoriamente em torno de zero, sem padrão
- **Padrão em forma de funil:** Heteroscedasticidade (variância não constante)
- **Padrão curvo:** Modelo não capturou relação não-linear
- **Outliers:** Pontos muito afastados (> 3 desvios padrão)

**Gráfico 2: Histograma**
- **Ideal:** Distribuição aproximadamente normal centrada em zero
- **Assimétrico:** Modelo tende a super/subestimar sistematicamente
- **Bimodal:** Pode indicar dois regimes diferentes nos dados

**Gráfico 3: Q-Q Plot**
- **Ideal:** Pontos seguem linha diagonal
- **Desvios nas pontas:** Caudas pesadas (outliers)
- **Desvios no centro:** Distribuição não-normal

**Estatísticas:**
- **Média ~0:** Resíduos não enviesados (desejável)
- **Desvio padrão:** Quanto menor, melhor o ajuste


<a id='comparacao'></a>
## **Comparação com Outros Modelos**

Para avaliar se a complexidade adicional de uma árvore de decisão é justificada, vamos compará-la com modelos mais simples como baseline.

In [ ]:
# =======================================================
# COMPARAÇÃO: ÁRVORE VS REGRESSÃO LINEAR VS SVR
# =======================================================

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# 1. Regressão Linear (baseline simples)
lr = LinearRegression()
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)

# 2. SVR RBF (baseline não-linear)
svr = SVR(kernel='rbf', C=10, epsilon=0.1, gamma='scale')
svr.fit(X_train, y_train)
pred_svr = svr.predict(X_test)

# 3. Árvore otimizada (já temos: best e pred_best)

# Comparar métricas
results = pd.DataFrame({
    'Modelo': ['Regressão Linear', 'SVR (RBF)', 'Árvore de Decisão'],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, pred_lr)),
        np.sqrt(mean_squared_error(y_test, pred_svr)),
        np.sqrt(mean_squared_error(y_test, pred_best))
    ],
    'MAE': [
        mean_absolute_error(y_test, pred_lr),
        mean_absolute_error(y_test, pred_svr),
        mean_absolute_error(y_test, pred_best)
    ],
    'R²': [
        r2_score(y_test, pred_lr),
        r2_score(y_test, pred_svr),
        r2_score(y_test, pred_best)
    ]
})

print("\n=== Comparação de Modelos ===")
print(results.to_string(index=False))

print("\n--- Análise ---")
best_model = results.loc[results['R²'].idxmax(), 'Modelo']
print(f"Melhor modelo (R²): {best_model}")

if results.loc[2, 'R²'] > results.loc[0, 'R²']:
    print("Árvore superou linear → dados contêm relações não-lineares e interações")
else:
    print("Linear competitivo → relação aproximadamente linear")
    
print("\nObservação: Árvores únicas são sensíveis a ruído.")
print("Para produção, considere ensembles (Random Forest, XGBoost).")


<a id='conclusoes'></a>
## **Conclusões e Dicas Práticas**

**Vantagens de Árvores de Decisão para Regressão**

1. **Interpretabilidade**: Modelo "caixa branca". Podemos visualizar regras de decisão (plot_tree, export_text).
2. **Sem premissas de distribuição**: Não assume linearidade nem normalidade de resíduos.
3. **Lida com interações naturalmente**: Captura relações não-lineares e interações sem engineering manual.
4. **Facilidade com dados mistos**: Features contínuas e categóricas sem transformações elaboradas (basta codificação simples).
5. **Rápida para treinar e prever**: Complexidade log(N) na predição quando balanceada.

**Desvantagens**

1. **Overfitting**: Sem regularização (max_depth, min_samples_leaf), tende a memorizar ruído.
2. **Instabilidade**: Pequenas mudanças nos dados alteram radicalmente a estrutura da árvore.
3. **Predições em degrau**: Árvore divide espaço em blocos. Dentro de cada bloco, predição é constante (média). Não suaviza bem limites.
4. **Pobre extrapolação**: Fora do range treinado, repetirá média da folha mais próxima (sem extrapolação linear).
5. **Tendência a viés em features com mais valores únicos**: Similar à classificação.

**Dicas Práticas**

- Comece com `max_depth` entre 3 e 7, depois ajuste com cross-validation.
- Use `min_samples_split` e `min_samples_leaf` para evitar folhas com poucos pontos (reduz variância).
- `criterion='absolute_error'` mais robusto se há outliers fortes.
- Observe resíduos: padrão forte indica má captura da estrutura ou necessidade de features adicionais.
- **Ensemble > Árvore Única**: Random Forest e Gradient Boosting (XGBoost, LightGBM) são superiores na prática ao combinar múltiplas árvores, reduzindo variância e melhorando generalização.

---

<-- [**Anterior: SVR**](02_svr.ipynb) | [**Próximo: Ensembles de Regressão**](04_ensembles_regressao.ipynb) -->
